In [1]:
!pip install trl

  Using cached trl-0.23.1-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-4.1.1-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.57.0-py3-none-any.whl.metadata (41 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached trl-0.23.1-py3-none-any.whl (564 kB)
Using cached datasets-4.1.1-py3-none-any.whl (503 kB)
Using cached multiprocess-0.70.16-py312-none-any.whl (146 kB)
Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (42.8 MB)
Using cached transformers-4.57.0-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstall

In [2]:
!pip install -U bitsandbytes

  Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.1-py3-none-manylinux_2_24_x86_64.whl (60.1 MB)


In [3]:
!pip install torch

In [4]:
!pip install peft

  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
Using cached peft-0.17.1-py3-none-any.whl (504 kB)


In [5]:
!pip install -U bitsandbytes

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./local_model_dir", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    "./local_model_dir",
    trust_remote_code=True,
)

# Get the device from the model (for quantized models, this is usually 'cuda' if available)
#device = next(model.parameters()).device

2025-10-06 13:15:39.978697: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759756540.004243    1338 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759756540.017163    1338 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-06 13:15:40.232298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear8bitLt(in_features=2560, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=2560, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=2560, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear8bitLt(in_features=2560, out_features=9728, bias=False)
          (up_proj): Linear8bitLt(in_features=2560, out_features=9728, bias=False)
          (down_proj): Linear8bitLt(in_features=9728, out_features=2560, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2560,)

In [8]:
tokenizer

Qwen2Tokenizer(name_or_path='./local_model_dir', vocab_size=151643, model_max_length=262144, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=

In [9]:
import pandas as pd

In [10]:
from datasets import load_dataset
from datasets import Dataset
import json

In [11]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0].role == \'system\' %}\n        {{- messages[0].content + \'\\n\\n\' }}\n    {%- endif %}\n    {{- "# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0].role == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0].content + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- endif %}\n{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}\n{%- for message in messa

In [12]:
train_df = pd.read_csv("./data/train_dataset.csv")

In [13]:
train_df = Dataset.from_pandas(train_df)

In [14]:
SYSTEM_PROMPT = "You are a helpful assistant that uses the provided tools to answer."

In [17]:
import json

def format_dataset_conversational(example):
    try:
        query = example['query']
        tools = json.loads(example['tools'])
        expected_calls = json.loads(example['answers'])

        # Serialize each tool call's arguments to a JSON string
        for call in expected_calls:
            if isinstance(call.get("arguments"), dict):
                call["arguments"] = json.dumps(call["arguments"])

        data = {
            "messages": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": query},
                {"role": "assistant", "tool_calls": expected_calls},
            ],
            "tools": tools
        }
        return data
    except Exception as e:
        print(f"Failed to process example: {e}")

In [19]:
conversational_train = train_df.map(format_dataset_conversational, remove_columns=train_df.column_names)

Map:   0%|          | 0/489 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoTokenizer
row = conversational_train[0]
rendered = tokenizer.apply_chat_template(
    row["messages"], tools=row["tools"], add_generation_prompt=False, tokenize=False
)
print(rendered)  # look for <tools>...</tools> and an assistant turn with <tool_call> JSON

<|im_start|>system
You are a helpful assistant that uses the provided tools to answer.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"description": "Computes the area of a circle given its radius.", "name": "circle_area", "parameters": {"contingency_table": null, "doubling_time": null, "email": null, "growth_rate": null, "initial_amount": null, "initial_population": null, "interest_rate": null, "intervals": null, "ip": null, "nested_list": null, "num": null, "num_years": null, "numbers": null, "radius": {"description": "The radius of the circle.", "type": "float"}, "significance_level": null, "time": null}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call><|im_end|>
<|im_start|>user
Calculate the area of circles 

In [21]:
conversational_train

Dataset({
    features: ['tools', 'messages'],
    num_rows: 489
})

In [22]:
from peft import LoraConfig
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "k_proj",
        "up_proj",
        "down_proj",
        "gate_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [23]:
model.add_adapter(lora_config, adapter_name="lora_1")

In [24]:
model.gradient_checkpointing_enable()

In [25]:
tokenizer.eos_token

'<|im_end|>'

In [27]:
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(
    output_dir="./qwen3-function-calling-ft-conversational",
    max_length=16384, 
    chat_template_path= "./local_model_dir/chat_template.jinja",
    eos_token=tokenizer.eos_token,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    packing=False, #changed this parameter
    fp16=True,
    bf16=False,
    report_to="none"
)

# ✅ SFTTrainer Configuration
trainer = SFTTrainer(
    model=model,
    train_dataset = conversational_train,
    args=training_args
)

# ✅ Start Training
trainer.train()

# ✅ Save Model
trainer.save_model("./qwen3-function-calling-ft-conversational")
tokenizer.save_pretrained("./qwen3-function-calling-ft-conversational")

Tokenizing train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/489 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.317400
20,1.266300
30,0.734900
40,0.521700
50,0.443500
60,0.372800
70,0.325000
80,0.272000
90,0.256000
100,0.224000


('./qwen3-function-calling-ft-conversational/tokenizer_config.json',
 './qwen3-function-calling-ft-conversational/special_tokens_map.json',
 './qwen3-function-calling-ft-conversational/chat_template.jinja',
 './qwen3-function-calling-ft-conversational/vocab.json',
 './qwen3-function-calling-ft-conversational/merges.txt',
 './qwen3-function-calling-ft-conversational/added_tokens.json')

In [86]:
tokenizer.pad_token

'<|endoftext|>'

In [88]:
type(tokenizer.chat_template)

str

In [28]:
import json
import re

def extract_tool_calls(text):
    # Extract tool calls using regex
    tool_call_pattern = r"<tool_call>(.*?)</tool_call>"
    matches = re.findall(tool_call_pattern, text, re.DOTALL)

    # Parse each match as JSON
    extracted_calls = []
    for match in matches:
        try:
            extracted_calls.append(json.loads(match))
        except json.JSONDecodeError:
            print(f"Failed to decode JSON: {match}")
            return False

    return extracted_calls

def compare_tool_calls(predicted, actual):
    actual_tool_calls = json.loads(actual)
    return actual_tool_calls == predicted

In [ ]:
train_df = pd.read_csv("./data/test_dataset.csv")